In [28]:
import cassandra
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, BatchStatement

from joblib import Parallel, delayed                                                                                                                                

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [29]:
index=False

In [30]:
"""
sudo docker run --name cassandra -p 127.0.0.1:9042:9042 -e CASSANDRA_CLUSTER_NAME=GangaTest -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra
"""
def cassandra_connection():
    """
    Connection object for Cassandra
    :return: session, cluster
    """
    cluster = Cluster(['127.0.0.1'], port=9042)
    session = cluster.connect()
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS fakehealthcareorg
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """)
    session.set_keyspace('fakehealthcareorg')
    return session, cluster


In [35]:
session, cluster = cassandra_connection()

# Creating the keyspace

In [37]:
session.execute("""create keyspace IF NOT EXISTS dev
... with replication = {'class':'SimpleStrategy','replication_factor':1};""")

In [38]:
session.execute("""use dev;""")

In [42]:
session.execute("""DROP TABLE IF EXISTS JOB""")
session.execute("""DROP TABLE IF EXISTS BLOB""")

In [43]:
jobs = pickle.load(open("../../data/rows.pkl", "rb"))[:size]
rows = []
for itr, job in enumerate(jobs):
    row = {}
    for header, value in zip(
        ['jid', 'status', 'name', 'subjobs', 'application',
        'backend', 'backend_actualCE', 'comment'],
        job
    ):
        row[header] = value

    row["jid"] = itr
    rows.append(row)
jobs = rows


_blobs = pickle.load(open("../../data/blobs.pkl", "rb"))[:size]
blobs = []
for i, blo in enumerate(_blobs):
    # FIXME: Dirty patch, for some reason any change to a single element fof blobs array, goes to all the elements. I decided to create copies after failing to debug this at that time.
    blob = copy.deepcopy(blo)
    blob["jid"] = i
    blobs.append(blob)

In [44]:
session.execute("""CREATE TABLE IF NOT EXISTS JOB(
    jid INT PRIMARY KEY, status varchar, name varchar, subjobs INT, application varchar,
    backend varchar, backend_actualCE varchar, comment varchar
)""")
session.execute("""
    CREATE TABLE IF NOT EXISTS BLOB(
        jid INT PRIMARY KEY, inputsandbox BLOB, outputsandbox BLOB, info BLOB,
        comment BLOB, time BLOB, application BLOB, backend BLOB,
        inputfiles BLOB, outputfiles BLOB, non_copyable_outputfiles BLOB,
        id BLOB, status BLOB, name BLOB, inputdir BLOB, outputdir BLOB, inputdata BLOB,
        outputdata BLOB, splitter BLOB, subjobs BLOB, master BLOB, postprocessors BLOB,
        virtualization BLOB, merger BLOB, do_auto_resubmit BLOB, metadata BLOB, been_queued BLOB,
        parallel_submit BLOB
    )
""")

In [45]:
query = session.prepare("""
    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (?,      ?,    ?,       ?,           ?,       ?,                ?,       ?)
""")

if jobs:
    with tqdm(total=len(jobs)) as progress:
        for i, row in enumerate(jobs):
            row[0] = i+1
            session.execute(query.bind(row))
            progress.update(1)


100%|██████████| 10000/10000 [00:09<00:00, 1091.83it/s]


In [46]:
%%time
future = session.execute_async("SELECT * FROM JOB")
rows = future.result()

CPU times: user 12.9 ms, sys: 4.27 ms, total: 17.1 ms
Wall time: 232 ms


In [47]:
%%time
len(list(rows))

CPU times: user 17.2 ms, sys: 6.92 ms, total: 24.2 ms
Wall time: 135 ms


10000

In [48]:
%%time
rows = session.execute("SELECT * FROM BLOB")

CPU times: user 5.36 ms, sys: 0 ns, total: 5.36 ms
Wall time: 8.45 ms


In [49]:
%%time
len(list(rows))

CPU times: user 25 µs, sys: 5 µs, total: 30 µs
Wall time: 36 µs


0